In [1]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F

sys.path.append("../")
from utils.sequence import load_data, get_vocab
from models.seq2seq import Seq2seq, Encoder, Decoder
from datasets.addition_dataset import AdditionDataset, Tokenizer

In [2]:
FILEPATH = "../datasets/addition.txt"
tokenizer = Tokenizer(FILEPATH)

In [3]:
tokenizer.questions

tensor([[ 2,  7,  0,  ...,  6,  0,  0],
        [ 6,  3,  0,  ...,  1,  8,  0],
        [ 8,  6,  0,  ...,  3,  0,  0],
        ...,
        [ 5,  9,  7,  ...,  7,  9,  0],
        [ 6, 10,  3,  ...,  8,  7,  0],
        [ 5,  4,  2,  ..., 10,  9,  0]])

In [8]:
# trainとvalidに分ける
sep_idx = 40000 
x_train = tokenizer.questions[:sep_idx, :] 
x_test = tokenizer.questions[sep_idx:, :] 
t_train = tokenizer.answers[:sep_idx, :] 
t_test = tokenizer.answers[sep_idx:, :] 
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)


torch.Size([40000, 7])
torch.Size([40000, 5])
torch.Size([10000, 7])
torch.Size([10000, 5])


In [9]:
# LSTMCellの練習
input_size = 10
hidden_size = 5
lstm = nn.LSTMCell(input_size, hidden_size)
x = torch.randn(2, 3, 10) # (timestep, batch, input_size)
hx = torch.randn(3, 5)
cx = torch.randn(3, 5)

for i in range(x.shape[0]):
    hx, cx = lstm(x[i], (hx, cx))
    print(hx, cx)

tensor([[ 0.0614, -0.2154, -0.1186,  0.2722, -0.0146],
        [-0.2839,  0.1138, -0.0473,  0.1592, -0.0729],
        [ 0.0354,  0.0124, -0.0187, -0.0232,  0.1013]], grad_fn=<MulBackward0>) tensor([[ 0.1190, -0.4301, -0.2519,  0.3787, -0.0299],
        [-0.4721,  0.2536, -0.2277,  0.4071, -0.2288],
        [ 0.1318,  0.0374, -0.0461, -0.1086,  0.3341]], grad_fn=<AddBackward0>)
tensor([[ 0.1024, -0.0628,  0.1266,  0.0673, -0.0263],
        [ 0.2182,  0.0849, -0.0692,  0.0820,  0.0174],
        [-0.1538,  0.0399,  0.1475,  0.1793,  0.1841]], grad_fn=<MulBackward0>) tensor([[ 0.1743, -0.1825,  0.3413,  0.3190, -0.0436],
        [ 0.4129,  0.3359, -0.2579,  0.3918,  0.0378],
        [-0.2132,  0.1153,  0.3730,  0.4073,  0.4795]], grad_fn=<AddBackward0>)


In [11]:
class CFG:
    num_layers = 2
    hidden_size = 16
    batch_size = 128
    epoch = 10

x_train_batch = x_train[:CFG.batch_size, :]
t_train_batch = t_train[:CFG.batch_size, :]

input_size= len(tokenizer.char2id)
encoder = Encoder(input_size, CFG.hidden_size, CFG.num_layers, input_size)
encoder_hidden = encoder.init_hidden(CFG.batch_size)
out, states = encoder(x_train_batch, encoder_hidden)
print(len(out))
print(out.shape)
# print(states[0].shape)
# print(states[1].shape)
decoder = Decoder(input_size, CFG.hidden_size, CFG.num_layers, input_size) 
# decoder_hidden = decoder.init_hidden(CFG.batch_size)
# states[0]: 隠れ状態
out, states = decoder(t_train_batch, states)
print("-"*36)
print(out.shape)
print(states[0].shape)
print(states[1].shape)

128
torch.Size([128, 7, 12])
------------------------------------
torch.Size([128, 5, 12])
torch.Size([2, 128, 12])
torch.Size([2, 128, 12])


In [16]:
t_train

tensor([[11, 10,  2,  0,  0],
        [11,  7,  6, 10,  0],
        [11, 10,  8,  0,  0],
        ...,
        [11,  6,  1,  5,  0],
        [11,  2,  1,  5,  4],
        [11,  2,  2,  3,  5]])

In [13]:
criterion = nn.CrossEntropyLoss()
print(torch.argmax(pred, axis=2).shape)
criterion(torch.argmax(pred, axis=2).float(), t_train_batch.float())

NameError: name 'pred' is not defined

In [ ]:
class CFG:
    num_layers = 2
    hidden_size = 16
    batch_size = 128
    epoch = 10

CFG.__dict__

mappingproxy({'__module__': '__main__',
              'num_layers': 2,
              'hidden_size': 16,
              'batch_size': 128,
              'epoch': 10,
              '__dict__': <attribute '__dict__' of 'CFG' objects>,
              '__weakref__': <attribute '__weakref__' of 'CFG' objects>,
              '__doc__': None})

In [63]:
x = torch.randn(16, 5, 12).reshape(-1, 12)
print(x.shape)
y = torch.ones(16, 5).reshape(-1, 1).int().squeeze()
print(y.shape)
z = x.index_select(1, y)[:, 0]
z.shape

torch.Size([80, 12])
torch.Size([80])


torch.Size([80])

In [72]:
x = torch.randn(3, 2)
x

tensor([[ 2.7748, -0.0990],
        [ 1.3342,  0.7899],
        [-0.7646, -0.6483]])

In [73]:
print(x.shape)
torch.flip(x,[0])

torch.Size([3, 2])


tensor([[-0.7646, -0.6483],
        [ 1.3342,  0.7899],
        [ 2.7748, -0.0990]])

In [78]:
x_train_batch

tensor([[ 2,  7,  0,  8,  6,  0,  0],
        [ 6,  3,  0,  7,  1,  8,  0],
        [ 8,  6,  0,  3,  3,  0,  0],
        [ 7,  4,  0,  3,  3,  0,  0],
        [ 8, 10,  6,  0,  4,  0,  0],
        [ 8,  1,  7,  0,  8, 10,  7],
        [ 9,  0,  5,  0,  0,  0,  0],
        [ 9,  5,  0,  4,  2,  8,  0],
        [10,  0,  4,  0,  0,  0,  0],
        [ 7,  0,  3,  0,  0,  0,  0],
        [ 2,  9,  0,  9,  0,  0,  0],
        [ 9,  6,  0,  6,  3,  0,  0],
        [10,  0,  2,  0,  0,  0,  0],
        [ 9,  0,  3,  1,  0,  0,  0],
        [ 6,  0,  4,  0,  0,  0,  0],
        [ 2,  0,  4,  0,  0,  0,  0],
        [ 6,  8,  0,  2,  1,  0,  0],
        [ 8,  0,  3,  1,  9,  0,  0],
        [ 8,  3,  4,  0,  9,  0,  0],
        [ 2,  5,  5,  0,  2,  5,  0],
        [ 6,  0,  8,  0,  0,  0,  0],
        [ 6,  0,  5,  7,  0,  0,  0],
        [10,  0,  2,  6,  0,  0,  0],
        [ 7,  3,  2,  0,  1,  0,  0],
        [ 9,  2,  0,  9, 10,  0,  0],
        [ 9,  5,  7,  0,  9,  5,  0],
        [ 3,

In [80]:

x_reverse = torch.flip(x_train_batch, [1])

In [85]:
print(x_train_batch.shape)
print(x_reverse.shape)
print(torch.cat((x_train_batch, x_reverse), axis=0).shape)

torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([256, 7])
